### Prediction on a nwe entry:

We receive from a bank employee the following informations:


{ "RevolvingUtilizationOfUnsecuredLines": 0.01703559, 
"NumberOfDependents": 1, 
"DebtRatio": 0, 
"age": 42, 
"NumberOfOpenCreditLinesAndLoans": 6, 
"NumberRealEstateLoansOrLines": 1, 
"NumberOfTime30-59DaysPastDueNotWorse": 1, 
"NumberOfTime60-89DaysPastDueNotWorse": 0, 
"NumberOfTimes90DaysLate": 0 }

- Create a new test set with new data from one input
- Do all modifications (same as the train_set)
- BigMl:
        - create sources
        - create datasets
        - train a model (ensemble) on the train_set
        - prediction on the new test_set
        - show the prediction



In [ ]:
from pandas import read_csv
import pandas as pd

In [ ]:
# Create dataframe with columns
df = pd.DataFrame(columns = ['Id',
 'RevolvingUtilizationOfUnsecuredLines',
 'age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'MonthlyIncome',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberOfTimes90DaysLate',
 'NumberRealEstateLoansOrLines',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfDependents',
 'SeriousDlqin2yrs',
 'NumberCreditLines'])
df

In [ ]:
# Create dataframe with new input
input = {'Id': [1],
         'RevolvingUtilizationOfUnsecuredLines': [0.01703559], 
         'age': [42], 
         'NumberOfTime30-59DaysPastDueNotWorse': [1], 
         'DebtRatio': [0], 
         'NumberOfOpenCreditLinesAndLoans': [6], 
         'NumberOfTimes90DaysLate': [0], 
         'NumberRealEstateLoansOrLines': [1], 
         'NumberOfTime60-89DaysPastDueNotWorse': [0], 
         'NumberOfDependents': [1],
       }
df_input = pd.DataFrame.from_dict(input)
df_input

In [ ]:
#Concatenate to fill the table with new data
df_exo = pd.concat([df, df_input], sort = False)
df_exo

In [ ]:
#Call the function for modifications and apply to exo dataframe
from ipynb.fs.full.Modifications_function import modification

df_change = modification(df_exo)
df_change

In [ ]:
export_csv = df_change.to_csv (r'exo_testset.csv', header=True)

In [ ]:
import bigml.api
from bigml.api import BigML
api= BigML (project='project/5d94a52e5a213962e2000331')

In [ ]:
# BigML prediction
source = api.create_source('fulltrain_dataframe.csv')
api.ok(source)
dataset_train = api.create_dataset(source)
api.ok(dataset_train)
test_source = api.create_source('exo_testset.csv')
api.ok(test_source)
dataset_test = api.create_dataset(test_source)
api.ok(dataset_test)
model = api.create_ensemble(dataset_train, {"objective_field":"SeriousDlqin2yrs"})
api.ok(model)
batch_prediction = api.create_batch_prediction(model, dataset_test, {"all_fields": True, "confidence": True, "probabilities": True})
api.ok(batch_prediction)
api.download_batch_prediction(batch_prediction, filename='exo_prediction.csv')
df_exo_final = pd.read_csv('exo_prediction.csv', index_col = 0)
df_exo_final

scikit-learn

In [ ]:
from pandas import read_csv
from pprint import pprint 
train_prepared = read_csv('fulltrain_dataframe.csv', index_col=0)
val_prepared = read_csv('exo_testset.csv', index_col=0)

In [ ]:
target_column = 'SeriousDlqin2yrs'
y_train = train_prepared[target_column].values
print(y_train)

X_train = train_prepared.drop(target_column, axis=1).values
pprint(X_train)

In [ ]:
y_val = val_prepared[target_column].values
X_val = val_prepared.drop(target_column, axis=1).values
print(y_val)
print(X_val)

In [ ]:
import pickle
from xgboost import XGBClassifier

filename = 'sklearn_model.sav'
loaded_model = pickle.load(open(filename, 'rb')) # Load the model already trained
result = loaded_model.predict(X_val)
print(result)

y_val_proba = loaded_model.predict_proba(X_val)
print(y_val_proba)
y_val_proba = y_val_proba[:,1]
print(y_val_proba)

In [ ]:
proba = 0.04850933
print("Probability =", proba)